In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import PIL
import numpy as np

import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

from model.data_loader import *
from model.generator_pix2pix import unet_pix2pix as build_generator
from model.discriminator_patchgan import patchgan70 as build_discriminator

In [3]:
def bce_loss(y_true, y_pred):
    """ 
    Binary cross entropy loss.
    - y_true: array of floats between 0 and 1
    - y_preds: sigmoid activations output from model
    """
    EPS = 1e-12
    x = -tf.reduce_mean((y_true * tf.log(y_pred + EPS)) + ((1-y_true) * tf.log(1-y_pred + EPS)))
    return x


def l1_loss(y_true, y_pred):
    """ L1 Loss with mean reduction per PyTorch default """
    # abs(targets - outputs) => 0
    return tf.reduce_mean(tf.abs(y_true - y_pred))


def d_loss_fn(model, x, y_true):
    """ """
    EPS = 1e-12
    y_pred = model([x[0], x[1]])
    loss_disc = bce_loss(y_true, y_pred)
    return loss_disc

def g_loss_fn(model, x, y_true, lambda_L1):
    """  """
    # abs(targets - outputs) => 0
    EPS = 1e-12
    g_pred, d_pred = model(x)
    loss_L1 = l1_loss(y_true[0], g_pred) * lambda_L1
    loss_gan = bce_loss(y_true[1], d_pred)
    loss_total = loss_gan + loss_L1
    return loss_total, loss_L1, loss_gan

In [4]:
train_pth = 'data/facades_processed/train'
input_sz = (256, 256, 3)

In [5]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    zoom_range=[0.8, 1.0],
    horizontal_flip=True,
    fill_mode='constant',
    data_format='channels_last',
    validation_split=0.0
)
train_loader = dataLoader(train_pth, train_generator, batch_sz=1, shuffle=True, img_sz=(256, 256))

In [6]:
# If discriminator.trainable == False, then batch norm won't be applied in gan
# Can do forward pass of gan with discriminator.trainable = True, 
# then only update generator's layers in backwards pass?

In [7]:
# build gan with discriminator.trainable = True
# ----------------------------------------------------

In [8]:
# Build discriminator
d_inputs, d_outputs = build_discriminator(input_size=input_sz, minibatch_std=False)
discriminator = Model(d_inputs, d_outputs, name='discriminator')

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# Build generator
g_inputs, g_outputs = build_generator(norm_type='batch', 
                                        input_size=input_sz, 
                                        output_channels=input_sz[-1])
generator = Model(g_inputs, g_outputs, name='generator')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
# Build gan
g_output_gan = generator(g_inputs)
d_output_gan = discriminator([g_output_gan, g_input])
gan = Model(g_input, [g_output_gan, d_output_gan], name='gan')

lr = 0.0002
lr_beta1 = 0.5
optimizer_g = tf.train.AdamOptimizer(learning_rate=lr, beta1=lr_beta1, beta2=0.999, epsilon=1e-08)


In [11]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
generator (Model)               (None, 256, 256, 3)  54423811    input_3[0][0]                    
__________________________________________________________________________________________________
discriminator (Model)           (None, 30, 30, 1)    2770497     generator[1][0]                  
                                                                 input_3[0][0]                    
Total params: 57,194,308
Trainable params: 57,182,660
Non-trainable params: 11,648
__________________________________________________________________________________________________


In [18]:
# forward pass through gan
# ----------------------------------------------------

In [19]:
batch_size = 1
discriminator_output_sz = (30, 30, 1)
lambda_L1 = 100.0
real = np.ones((batch_size, ) + discriminator_output_sz)   # real => 1

In [20]:
inputs, targets = next(train_loader)

with tf.GradientTape() as tape:
    g_loss_total, g_loss_L1, g_loss_gan = g_loss_fn(gan, inputs, [targets, real], lambda_L1)


In [32]:
# apply gradients only to generator
grads = tape.gradient(g_loss_total, gan.trainable_variables[:43])
optimizer_g.apply_gradients(zip(grads, gan.trainable_variables[:43]),
                                    global_step=tf.train.get_or_create_global_step())

In [12]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
generator (Model)               (None, 256, 256, 3)  54423811    input_3[0][0]                    
__________________________________________________________________________________________________
discriminator (Model)           (None, 30, 30, 1)    2770497     generator[1][0]                  
                                                                 input_3[0][0]                    
Total params: 57,194,308
Trainable params: 57,182,660
Non-trainable params: 11,648
__________________________________________________________________________________________________


In [17]:
len(generator.trainable_variables)

43

In [ ]:
# train discriminator

In [18]:
discriminator.trainable_variables

[<tf.Variable 'conv2d_4/kernel:0' shape=(4, 4, 6, 64) dtype=float32, numpy=
 array([[[[-7.56909788e-01, -1.03965807e+00, -1.31568813e+00, ...,
            1.96749121e-01,  2.90909719e+00, -1.01860571e+00],
          [ 3.39462161e-01,  4.12167609e-01,  1.09432149e+00, ...,
           -1.49181068e+00,  4.29213434e-01,  1.39720130e+00],
          [-2.69817770e-01, -5.70266247e-01, -1.22349453e+00, ...,
            9.29870248e-01, -1.24686480e+00,  1.11852098e+00],
          [ 4.30024415e-01, -3.06044817e-01,  2.43814677e-01, ...,
            1.58984387e+00,  3.13457012e-01, -5.91355205e-01],
          [-1.84159040e+00, -1.79527491e-01, -6.62636936e-01, ...,
           -1.25655219e-01, -5.97417176e-01,  5.71003735e-01],
          [-1.17288634e-01, -1.92407846e-01, -2.30240941e-01, ...,
           -1.30925786e+00,  2.78522223e-01, -8.31856847e-01]],
 
         [[ 1.64983392e-01,  1.57959175e+00, -3.87983978e-01, ...,
            1.29050100e+00,  2.01108432e+00,  1.30573773e+00],
          [

In [3]:
optimizer_g = tf.train.AdamOptimizer(learning_rate=0.2, beta1=0.9, beta2=0.999, epsilon=1e-08)

In [4]:
dir(optimizer_g)

['GATE_GRAPH',
 'GATE_NONE',
 'GATE_OP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_variable_with_custom_getter',
 '_apply_dense',
 '_apply_sparse',
 '_apply_sparse_duplicate_indices',
 '_apply_sparse_shared',
 '_assert_valid_dtypes',
 '_beta1',
 '_beta1_t',
 '_beta2',
 '_beta2_t',
 '_call_if_callable',
 '_checkpoint_dependencies',
 '_create_non_slot_variable',
 '_create_or_restore_slot_variable',
 '_create_slots',
 '_deferred_dependencies',
 '_deferred_slot_restorations',
 '_distributed_apply',
 '_epsilon',
 '_epsilon_t',
 '_finish',
 '_gather_saveables_for_checkpoint',
 '_get_beta_accumulators',
 '_get_non_slot_variable',
 '_get_or_make_slot',
 '_get_or_ma

In [8]:
optimizer_g._lr_t